# Ejercicio 2

## Implementacion de una red neuronal que incorpora una regresión lineal.

### Importamos los datos en un dataframe

In [ ]:
import pandas as pd
df = pd.read_csv('insurance.csv')

Breve Análisis del dataset
Mostramos primero algunos valores de nuestro dataset.

Luego los nombres de las columnas de nuestro dataframe

Y para finalizar tambien mostramos el tamaño del dataframe con el que trabajaremos

In [ ]:
print("Next the first rows of our dataset are showed: \n")
print(df.head())
print("\nThe colums of our dataset refears to: \n"+str(df.columns) + "\n")
print("the size of the dataframe is: \n " + str(df.shape) + "\n")

Con la siguiente linea nos fijamos los tipos de datos que tenemos presentes en cada columna



In [ ]:
df.info()

Verificamos la cantidad de datos nulos presentes en nuestro dataframe.

In [ ]:
df.isnull().sum()

Podemos utilizar el método describe para examinar como son los valores que se presentan dentro del dataframe.

In [ ]:
df.describe()

### Breve análisis del dataframe

De ver los valores que devuelve describe podemos darnos cuenta que como los valores minimos de edad no son 0 entonces no hay datos nulos ni ceros que podrían se considerados invalidos en esa columna, de hecho el valor maximo es 64 por lo que podríamos plantear a priori que no es necesario una limpieza de los datos para esa columna.
Luego en el BMI vemos que no existen valores iguales a 0 de hecho el valor minimo es 15 lo cual es un valor posible y el valor maximo es 53 lo cual es un valor muy alto pero al ser el valor mas alto de nuestro dataframe podríamos considerar que esta "bien" y que luego será el diagrama de caja el que nos termine de definir si pertenece o no al grupo de los outlayers, pero en principio uno podría afirmar que el valor es posible en un humano, muy poco saludable pero posible. 
La cantidad de hijos tambien es un parámetro que no presenta valores nulos ni invalidos, debido a que el minimo es cero el cual es un valor perfectamente posible y el valor maximo es 5 lo cual tambien parecería ser un valor esperado, por lo tanto no es necesario realizar ningun tipo de limpieza de datos sobre el mismo.
Los charges que son sobrados por la atencion medica recibida en principio tampoco tienen valores nulos ni ceros lo cual implica que no hay que realizar una limpieza de los datos, pero es de notar que la varianza es elevada, el valor minimo es cercano a 1000 y el valor maximo es cercano a 60.000 lo cual nos invita a pensar en como se podrían limpiar estas métricas, no porque las mismas estén mal tipeadas, sino mas bien porque es probable que nuestro modelo no requiera saber la informacion de atenciones medicas tan bajas de costo o tan altas, lo decidirá en parte el diagrama de caja y lo bien que trabae nuestro modelo.

### Datos que faltan

Existen ciertas columnas que el metodo describe no es capaz de computar debido a que las mismas son columnas que se encuentran descriptas por strings. Para manejar estos datos lo que haremos será ver como estan compuestos los mismos, para eso podemos observar que en la columna "smoker" lo que se indica es si la persona es fumadora o no, esto se indica mediante las palabras "yes" o "no". Por lo tanto lo que podemos hacer es reemplazar todas las apariciones de yes por un 1 y todas las operaciones de no por un 0, esto lo podemos hacer de esta manera ya que es una variable categórica que presenta solo 2 estados posibles por lo que al asignarle un numero a cada una no le añadimos un peso consigo, no implica un ordenamiento de las categorías.

Un caso similar ocurre para la columna sex que se clasifica en male or female.

En cambio para el caso de los barrios ahí tenemos otro problema distinto debido a que ahora es una distribucion categótica pero con varias categorías, por lo que no podemos proceder de la misma manera que antes.

A continuacion se muestra una manera de saber los posibles valores que pueden tomar cada una de estas columnas.

In [ ]:
# count occurances a particular column
occur = df.groupby(['smoker']).size()
  
# diplay occurances of a particular column
display(occur)

In [ ]:
# count occurances a particular column
occur = df.groupby(['sex']).size()
  
# diplay occurances of a particular column
display(occur)

In [ ]:
# count occurances a particular column
occur = df.groupby(['region']).size()
  
# diplay occurances of a particular column
display(occur)

Primero resolvamos el problema de la columna smokers: 

Para esto lo que tenemos que hacer es reemplazar todos los yes por 1 y todos los no por 0:

In [ ]:
df[['smoker']] = df[['smoker']].replace('yes',1)
df[['smoker']] = df[['smoker']].replace('no',0)

Para comprobar que haya funcionado como esperamos podemos comprobar a continuacion que describe ahora sí contempla la columna smoker y que la misma presenta una gran cantidad de no (0) y poca cantidad de yes (1) (la proporcion esta dada por la media que es aproximadamente 0.205), por lo tanto solo 1/5 de las personas son fumador


In [ ]:
df.describe()

Una vez realizado el estudio de la columna smoker procedemos a continuacion a realizar la limpieza de la columna sex, la mismas se clasifica en male o female, por lo que introduciremos el numero 1 en caso de que se trate de un hombre y el numero 0 si se tratara de una mujer:


In [ ]:
df[['sex']] = df[['sex']].replace('male',1)
df[['sex']] = df[['sex']].replace('female',0)

Para comprobar que todo funcionó de manera correcta podemos checkear el resultado de la instruccion describe:


In [ ]:
df.describe()

De esta manera podemos notar que al encontrarse la media en torno al valor 0.5 podemos decir que los datos estan balanceados para esa columna.


Para el análisis de la variable en la columna region, como el numero de categorías es pequeño (4), podemos plantear una clasificacion mediante One Hot Encoding en lugar de utilizar embebbing, ya que de esta manera el análisis es simple y la cantidad de variables que agregamos no es tan grande como para necesitar realizar el cambio a embebbing.

A continuación se detalla el procedimiento por el cual se realiza la codificacion mediante One Hot Encoding:

In [ ]:
# Get one hot encoding of columns Bs
one_hot = pd.get_dummies(df['region'])
# Drop column B as it is now encoded
df = df.drop('region',axis = 1)
# Join the encoded df
df = df.join(one_hot)

A continuación mostramos como quedó el dataset:


In [ ]:
df.head()

Simplemente por una cuestion de orden pondremos la variable que no interesa hallar como la ultima columna del dataframe:

In [ ]:
df = df[['age', 'sex', 'bmi', 'children', 'smoker', 'northeast', 'northwest', 'southeast', 'southwest', 'charges']]

In [ ]:
df.head()

A continuación podemos ampliar la informacion aportada en esta etapa con algunos graficos de histogramas que nos permitan comprobar las distribuciones de edad bmi e hijos de las personas que formaron parte de este estudio.

In [ ]:
# Se importan librerías para graficar.
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# dfToPlot = df[['age', 'sex', 'bmi', 'children', 'smoker', 'charges']]
# fig, axes = plt.subplots(nrows=2, ncols=3)
# for i, col in enumerate(dfToPlot.columns):
#     sns.histplot(dfToPlot[col], ax=axes[int(i/3)][int(i%3)], bins=10)
# axes[0][0].figure.set_size_inches(18, 10)
# fig.suptitle('Histograms', fontsize=20)
# plt.tight_layout()
# plt.show()

### De los gráficos anteriores podemos sacar algunas conclusiones: 

Las edades de los encuestados se encuentran bastante bien niveladas, sin presentar edades con valores muy superiores a otras lo cual es algo bueno para nuestro análisis. El sexo de las personas encuestadas tambien es algo que se enceuntra bien nivelado, presentandose la misma cantidad de personas mujeres que de hombres. La distribución de BMI en las personas se concentra en un valor cercano a 30 y disminuye hacia ambos lados. 

Las probabilidades decrecen al aumentar la cantidad de hijos. 

Existen mas personas no fumadoras que fumadoras en nuestro estudio.

Y por ultimo tambien podemos notar que el numero de pagos disminuye a medida que aumenta el valor del mismo, por lo que son muy pocos frecuentes los casos en los que se requiera un pago cercano al valor 60.000.

Para continuar con el análisis gráfico de la situación, debajo se encuentran los diagramas de cajas para las diferentes variables que conforman nuestro estudio, no se tienen en cuenta para ello las distribuciones con respecto a la region de las personas que se sabe que esta bastante bien nivelada la cantidad de personas que pertenecen a cada zona. Tampoco se grafican los diagramas de caja para las variables binomiales porque no nos aporta ninguna informacion este tipo de gráficos. Por ultimo se aclara que el diagrama para los charges es decir para los gastos de la atención médica se incorporan en un grafico aparte debido a que los valores son mucho mayores a los de las demás categorías.

In [ ]:
# f, ax = plt.subplots(figsize=(20, 10))
# plt.ylabel('Variables')
# plt.title("Boxplots")
# ax = sns.boxplot(data = dfToPlot[['age', 'bmi', 'children']], 
#   orient = 'h', 
#   palette = 'Set2')

In [ ]:
# f, ax = plt.subplots(figsize=(20, 10))
# plt.ylabel('Variables')
# plt.title("Boxplots")
# ax = sns.boxplot(data = dfToPlot[['charges']], 
#   orient = 'h', 
#   palette = 'Set2')

### Análisis de los boxplots:

De los diagramas de caja podemos notar que existen algunos outliers con respecto a los valores de BMI que se deberá verificar como proceder con ellos (algo que se realiza mas adelante en el análisis) ademas se deberá tomar una decisión respecto de los puntos que conforman el charge es decir el gasto de la atención medica que son outliers, debido a que para esta variable el numero de outliers es muy grande.

Por nuestro conocimiento del problema podemos deducir que la varianza en el precio de una atencion medica puede deberse a diferentes factores. De hecho nosotros mismos tenemos que diseñar la clasificación de manera tal de encontrar dichos factores, pero resulta comprensible que la varianza en el precio sea alta por lo que no consideramos que haga falta retirar dichos outliers de nuestro sistema, aunque de todas maneras si resulta indispensable considerar que existen debido a que la varianza en el pago es grande.

### Grafico de correlacion de variables

A continuación se muestran los diagramas de correlacion de variables que nos permiten tener una idea de la relacion entre las mismas.

In [ ]:
# #print a pairplot to check the relationships between strongly correlated features
# pp = sns.pairplot(df[['age', 'sex', 'bmi', 'children', 'smoker', 'charges']])
# pp = pp.map_lower(sns.regplot)
# pp = pp.map_upper(sns.kdeplot);

### Análisis del gráfico de correlacion de variables:

Del gráfico anterior se pueden ver datos muy relevantes, si analizamos la fila correspondiente a la edad, veremos que no presenta correlacion frente al sexo lo cual nos habla de un estudio en principio bien realizado, tampoco varia demaisado con respecto al valor del bmi ni de la cantidad de hijos y la distribucion de edad para fumadores y no fumadores parece ser en principio muy parecida. Pero de todas maneras se puede ver de manera clara que existe una correlacion entre la edad de la persona y el charge se puede ver como los datos se ubican de forma tal que generan una pendiente notoria.

De analizar la fila para sex podemos notar que la correlacion de sexo con bmi es muy pequeña, existe una distribucion parecida para las personas de sexo masculino o femenino, presentandose sí los valores de mayor BMI en hombres. 

Con respecto a la realcion entre seo y cantidad de hijos las distribuciones son muy similares y lo mismo ocurre con los fumadores, presentandose una ligera mayoria en los hombres fumadores respecto de las mujeres fumadoras, pero no es tan grande la diferencia. El sexo influye pero poco en los gastos de salud, presentandose un aumento del costo en el caso de los hombres.

Con respecto a la fila de bmi podemos notar que existe una correlacion entre el bmi y el costo de salud medica, debido a que valores mayores de BMI tienden a presentar valores mayores de costos médicos.

luego respecto de la cantidad de hijos se puede apreciar que las personas con mayor cantidad de hijos presentan valores menores de gastos medicos, algo que puede ser entendido debido a que estas personas quiza deban destinar parte de su dinero a sus familias en lugar de destinarlo a gastos medicos.

Por ultimo la relacion entre los costos y si la persona es o no fumadora es muy grande, la correlacion existe de manera muy notoria en este caso, por lo que se ha de tener en cuenta para el análisis.

### Matriz de correlacion:

Por último y para finalizar con el análisis de la situción se presenta a continuación una matriz que muestra la correlacion de las variables involucradas:

In [ ]:
# correlation = df.corr()
# sns.set(rc={'figure.figsize':(12,8)})
# sns.heatmap(correlation,annot = True, linewidth=1, linecolor='w', cmap="Reds")

### Análisis de la matriz de correlacion

Podemos ver en la matriz de correlacion que se encuentra arriba que los valores de correlacion se corresponden a las deducciónes que pudimos realizar sobre el modelo en base a los datos considerados de a pares en los diagramas de puntos. De esta manera podemos apreciar la fuerte correlacion que existe entre el precio pagado y el hecho de que la persona sea o no fumadora ademas de la influencia del bmi y la edad en el dinero que se destina a la salud. 

Una correalcion interesante que no podiamos ver en los graficos anteriores y ahora sí es la que existe entre el BMI y la region en la que viven las personas, principalmente para el caso de la region southeast en la cual la correlacion es muy fuerte respecto del BMI, este dato tal vez no es tan relevante para nuestro caso, pero podría servirnos si tuvieramos que postular valores si hubiera datos faltantes introducidos por el usuario.

De esta manera hemos finalizado la etapa de limpieza y ordenamiento de los datos que nos permitirá a partir de ahora trabaja de manera mas estructurada con los mismos.




